In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator


## Download Dataset


A dataset containing 18632 images of infected as well as healthy leaf images are in the train_images.The test_images folder contains 3 test images.Create a tf.data.Dataset for training and validation using the tf.keras.preprocessing.image_dataset_from_directory utility. 

### Count the Number of images in Trainset and Validationset

In [ ]:
from glob import *
import pathlib
train_dir = '../input/plant-pathology-2021-fgvc8/train_images/'
test_dir = '../input/plant-pathology-2021-fgvc8/test_images/'
train_dir = pathlib.Path(train_dir)
test_dir = pathlib.Path(test_dir)
print("Trainset Count:", len(list(train_dir.glob('*.jpg'))))
print("TestSet Count:", len(list(test_dir.glob('*.jpg'))))

### Print the first image in the training set

In [ ]:
Infected_leaf = list(train_dir.glob('*.jpg'))
PIL.Image.open(str(Infected_leaf[0]))

Initially I tried to use the tf.keras.preprocessing.image_dataset_from_directory to create the directories for the train and test images. As the file names are numbers Keras API is considering it in float32.To resolve this problem we are doing the following steps below. So I decided to use the ImageDataGenerator. For this I nees to get the image names from the train.csv file.

So first I will get the train.csv file and convert it into dataframe and print the dataframe head(). There are two colums, image id and the target classes, the labels  a space delimited list of all diseases found in the image. Unhealthy leaves with too many diseases to classify visually will have the complex class, and may also have a subset of the diseases identified.

In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df.head()

There are 12 target classes. Each images belong to one of these classes.


In [ ]:
df['labels'].unique()

Now, let us see the distribution of data i.e.how many images belongs to each disease and how many images are categorised as healthy and how many leaves with multiple disease are categorized to complex. It seems the class 'scabe' has the most number of images. The data is highly imbalanced.

In [ ]:
df.labels.value_counts()

Let us plot and see the distribution of data. You can clearly view how imbalanced the dataset is.Also there are classes which holds the subset of diseases. For example rust frog_eye_leaf_spot complex,  scab frog_eye_leaf_spot complex etc. This is a multi label classification problem as there are more than one disease in one image.

In [ ]:
df['labels'] = df['labels'].astype(str)
plt.figure(figsize=(8,5))
sns.countplot(data = df,y='labels')

Lets view one sample from each class. 

In [ ]:
import cv2
train_dir = '../input/plant-pathology-2021-fgvc8/train_images/'
test_dir = '../input/plant-pathology-2021-fgvc8/test_images/'
for label in list(df['labels'].unique()):
    fig, ax = plt.subplots(1, 1, figsize=(5, 3))
    idx = df[df['labels']==label].index[0]
    image = cv2.imread(train_dir+df.loc[idx, 'image'])
    image =cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    ax.imshow(image)
    ax.set_title(label)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

## Finding Duplicates

In [ ]:
Image.open((train_dir+df['image'])[0])

In [ ]:
from PIL import Image
import imagehash

# image_fns : List of training image files
img_hashes = {}
i=0
for img in list(df['image']):
#for img_fn in sorted(image_fns):
    #idx = df[df['labels']==label].index[i]
    #i=i+1
    #image = cv2.imread(train_dir+df.loc[idx, 'image'])
    #image =cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = train_dir+img
    hash = imagehash.average_hash(Image.open(image))
    if hash in img_hashes:
        print( '{} duplicate of {}'.format(image, img_hashes[hash]) )
    else:
        img_hashes[hash] = image